<a href="https://colab.research.google.com/github/momna763/NLP/blob/main/IMDB%20Review%20Sentiment%20Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:

# Step 2: Load the Dataset
import pandas as pd

# Load the dataset (make sure to upload the CSV file to Colab)
df = pd.read_csv('IMDBDataset.csv')

# Preprocess the sentiment column
df['sentiment'] = df['class'].map({'positive': 1, 'negative': 0})

# Verify the conversion
print("Sentiment values after mapping:", df['sentiment'].unique())

# Preprocess the reviews
import re
from bs4 import BeautifulSoup

def preprocess_text(text):
    # Check if the text contains HTML and parse if necessary
    if '<' in text and '>' in text:
        text = BeautifulSoup(text, "html.parser").get_text()

    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply preprocessing to the review column
df['review'] = df['review'].apply(preprocess_text)
# Step 4: Feature Extraction
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Split the data into training and test sets
X = df['review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data to numerical data using Count Vectorization
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Step 5: Train the Naïve Bayes Model
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

# Step 6: Evaluate the Model
from sklearn.metrics import accuracy_score, classification_report

# Predict on the test set
y_pred = model.predict(X_test_vectorized)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Print classification report
print(classification_report(y_test, y_pred))

# Step 7: Build a User Interface
import gradio as gr

def predict_sentiment(review):
    # Preprocess the review
    review_cleaned = preprocess_text(review)
    review_vectorized = vectorizer.transform([review_cleaned])
    prediction = model.predict(review_vectorized)
    return 'Positive' if prediction[0] == 1 else 'Negative'

# Create a Gradio interface
iface = gr.Interface(fn=predict_sentiment, inputs="text", outputs="text",
                     title="IMDB Review Sentiment Classifier",
                     description="Enter a movie review to predict its sentiment (Positive/Negative).")
iface.launch()


Sentiment values after mapping: [1 0]
Accuracy: 0.85
              precision    recall  f1-score   support

           0       0.83      0.88      0.85      4961
           1       0.87      0.82      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9792cfc67e2a126a2d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
